# ETL Entity and Topic Extraction

This notebook will aim to extract important topics from documents all documents. 
The output will be a cleaned dataset containing all text content for analysis.

In [1]:
import sys

sys.path.append(r'c:\users\owner\appdata\roaming\python\python37\site-packages')
sys.path.append(r'..\tools')


import scispacy
from scispacy.abbreviation import AbbreviationDetector
import spacy
import pandas as pd
import json
import os
import re
from tqdm.notebook import tqdm
from rake_nltk import Rake
from spacy.matcher import Matcher 
import en_core_web_lg

from nltk.stem import WordNetLemmatizer 
import nltk
                
from loading.document import document
from NER.tripple_extract import get_entity_pairs
from concurrent import futures

from multiprocessing import Pool





## Identify and load file directories

In [2]:
directories = ['../data/biorxiv_medrxiv/', '../data/comm_use_subset/','../data/custom_license/', '../data/noncomm_use_subset/']

In [3]:
files = []
for directory in directories:
    for (dirpath, dirnames, filenames) in os.walk(directory):
        filenames = [names for names in filenames if '.json' in names]
        if filenames != []:
            files.append({'dirpath':dirpath, 'filenames':filenames})
        

### Initial string cleaning

**TODO: improving text cleaning will provide fundamental improvements.**

### Simple topic extraction can be used to cluster documents. 

In [4]:
def phrases_extract(text):
    r = Rake()
    r.extract_keywords_from_text(text)
    key_phrase = r.get_ranked_phrases() # To get keyword phrases ranked highest to lowest.
    
    return key_phrase

## Keyword Search

Using keywords to link publication to Kaggle tasks.

In [5]:
with open('../tasks/keywords.json') as json_file:
    keywords = json.load(json_file)

def identify_tasks(text, keywords):
    text = lemmatizer.lemmatize(text)
    task_list=[]
    for k,v in keywords.items(): 
        if any(x in text for x in v['keywords']):
            task_list.append(k)  
    return task_list


## Apply each of the techniques outline above to each publication and aggrigate findings.

In [6]:
directory_list =[]
file_list = []
for file in files:
    directory = file['dirpath']
    for filenames in file['filenames']:
        directory_list.append(directory)
        file_list.append(filenames)

In [13]:
def process_document(location):
    directory, file = location
    file_path = directory + '/' + file
    file_save = directory + '_extract/' + file

    if os.path.isfile(file_save):
        pass
    else:
        try:
            pub = document(file_path)
            pub.clean_text()
            try:
                pairs, ents = get_entity_pairs(pub.abstract)
                pub.abstract_tripples = pairs.to_json()
            except:
                pub.abstract_tripples = '{"subject":{},"relation":{},"object":{},"subject_type":{},"object_type":{}}'

            try:
                pairs, ents = get_entity_pairs(pub.text)
                pub.text_tripples = pairs.to_json()
                pub.entities = ents 
            except:
                pub.text_tripples = '{"subject":{},"relation":{},"object":{},"subject_type":{},"object_type":{}}'

            try:
                pub.save(file_save)
            except:
                pass
        except:
            pass

In [ ]:
for item in tqdm(list(zip(directory_list, file_list))):
    process_document(item)
